<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/scare_quote_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import sys
import scipy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en import English
import numpy as np

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')
#TODO
# fix -- change to scare quotes folder
%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/1_Data Prep
%ls


scare_quote_data_prep.ipynb


BigQuery

In [ ]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='detecting-bias-mimic-iii')

icd_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.diagnoses_icd
                LIMIT 1000''')

icd = client.query(icd_query).to_dataframe()



ICD codes

In [ ]:


#Filter for ICD codes for 282.60-282.69, referring to sickle cell types w/wo crisis
#2824 for thalassemia w + w/o crisis (282.41-282.42)
#SCD: 2826,2824
#Chronic Pain 3382
#Opioid dependencies: 3040,3047 (combo),
#HIV/AIDS ^042$

icds_of_interest = icd[icd['ICD9_CODE'].str.contains('2826|2824|3040|3047|3382|^042$', na=False)]
# Issues here-- will cause more leading numbers
print(icds_of_interest.head())

icds_of_interest.info()

#patients_unique = icds_of_interest['SUBJECT_ID'].drop_duplicates()


Notes and patients load

In [ ]:
note_query = ('''SELECT *
                FROM physionet-data.mimiciii_notes.noteevents
                LIMIT 50000''')

NOTES = client.query(note_query).to_dataframe()

peek_notes = NOTES.head()
print(peek_notes)

## Patients

patient_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.patients''')

PATIENTS = client.query(patient_query).to_dataframe()

peek_patients = PATIENTS.head()
print(peek_patients)

biased_notes_patients_corpus = NOTES.merge(PATIENTS, on = 'SUBJECT_ID')

biased_notes_patients_corpus.info()
# Find unique chart types and decide to limit them
    # Remove Radiology, ECG, Respiratory, Echo notes
biased_notes_patients_corpus_filtered = biased_notes_patients_corpus[biased_notes_patients_corpus["CATEGORY"].str.contains("Radiology|ECG|Respiratory|Echo")==False]
# biased_notes_patients_corpus_filtered.to_csv("biased_notes_patients_corpus_filtered.csv")



Explode by sentence, regex quote match

In [ ]:
full_dataframe = biased_notes_patients_corpus_filtered
# Tokenize by sentence
nlp = English()  # just the language with no model

nlp.add_pipe('sentencizer')
full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])
full_dataframe = full_dataframe.explode("Sentence", ignore_index=True)
full_dataframe.rename(columns={"Unnamed: 0": "ROW_ID_new"}, inplace=True)
full_dataframe.index.name = "Sentence ID"

full_dataframe['Sentence'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)




In [4]:
# Read in tokenized data frame
full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")



<ipython-input-4-9be9d7fd8d2f>:2: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")


In [7]:

regex = "\"(.+?)\""
# Add & needs to include patient | pt , lowercase match
quoted_dataframe = full_dataframe.loc[full_dataframe['Sentence'].str.contains(regex)]

quoted_dataframe["scare_quote"] = ""
quoted_dataframe["annotator_comments"] = ""
# 3842 quoted charts sentences total of 1,510,650 sentences
quoted_dataframe2 = quoted_dataframe.drop_duplicates(subset=['Sentence'])
quoted_dataframe2.to_csv("quoted_dataframe_annotate.csv")



<ipython-input-7-55c29e1450bc>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  quoted_dataframe = full_dataframe.loc[full_dataframe['Sentence'].str.contains(regex)]


KeyboardInterrupt: ignored

Explore the Data


In [ ]:
from google.colab import runtime
runtime.unassign()